In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import nest_asyncio
import warnings

_ = load_dotenv(find_dotenv())
nest_asyncio.apply()
warnings.filterwarnings("ignore")

In [3]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding

import sys

__curdir__ = os.getcwd()
sys.path.append(
    "../tools"
)

from calculator_tools import get_calculator_tool
from data_analysis_tools import get_da_tools
from fundamental_analysis_tools import get_fa_tools
from search_tools import get_tavily_tool
from technical_analysis_tools import get_ta_tools  

Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = GeminiEmbedding()

In [4]:
calculator_tool = get_calculator_tool()
da_tool = get_da_tools()
fa_tool = get_fa_tools()
search_tool = get_tavily_tool()
ta_tool = get_ta_tools()

In [5]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.tools import FunctionTool
from typing import List, Annotated, Literal

def create_agent(
    tools: Annotated[List[FunctionTool], "List of tools"]
):
    agent_worker = FunctionCallingAgentWorker.from_tools(tools)
    return agent_worker.as_agent()

DataAnalyst = create_agent(tools = da_tool)
FundamentalAnalyst = create_agent(tools=[fa_tool])
TechnicalAnalyst = create_agent(tools = ta_tool)
Researcher = create_agent(tools=[*ta_tool, *calculator_tool])

In [12]:
agents = {
    "DataAnalyst": DataAnalyst,
    "FundamentalAnalyst": FundamentalAnalyst,
    "TechnicalAnalyst":TechnicalAnalyst,
    "Researcher": Researcher,
}

## Create Workflow

### Create router program

In [8]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.bridge.pydantic import BaseModel

VALID_AGENTS = Literal["DataAnalyst", "FundamentalAnalyst", "TechnicalAnalyst", "Researcher"]

class AgentRoute(BaseModel):
    """Defines which agent to route the tasks to"""
    
    agents: Annotated[List[VALID_AGENTS], "The valid agents that can answer this question."]
    
prompt_template_str = """\
Think of the agents that can best answer the user query: {query}.
"""

program = LLMTextCompletionProgram.from_defaults(
    output_cls = AgentRoute,
    prompt_template_str = prompt_template_str,
    verbose = True,
    llm = Gemini(model="models/gemini-1.5-flash")
)

In [9]:
output = program(query="Should I invest in Illumina shares")
print(output.agents)

['FundamentalAnalyst', 'TechnicalAnalyst', 'Researcher']


### Create workflow

In [10]:
from llama_index.core.workflow import (
    Event,
    Context,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

In [18]:
class GetAgentsEvent(Event):
    """Event to get agents"""
    
    task: str
    agents: Annotated[List[VALID_AGENTS], "The agents that can answer the question"]

class AggregateResultsEvent(Event):
    """Event to aggregate results from individual agents"""
    
    task: str
    aggregate_result: str

# class ConsolidateResultsEvent(Event):
#     """Event to consolidate aggregated results"""
    
#     task: str
#     final_result: str

In [19]:
from llama_index.core.prompts import PromptTemplate

CONSOLIDATE_TEMPLATE = PromptTemplate(
    "Craft a final answer to answer the question {task} using the following intermediate results: {aggregated_results}"
)

In [20]:
class InvestmentWorkflow(Workflow):
    """Workflow for investment decisions"""
    
    @step(pass_context=True)
    async def get_agents(
        self, ctx: Context, ev: StartEvent
    ) -> GetAgentsEvent:
        """Step to route to agents"""
        
        task = ev.get("task")
        program = LLMTextCompletionProgram.from_defaults(
            output_cls = AgentRoute,
            prompt_template_str = prompt_template_str,
            verbose = True,
            llm = Gemini(model="models/gemini-1.5-flash")
        )
        ctx.agents = agents
        ctx.llm = Gemini(model="models/gemini-1.5-flash")
        
        if task is None:
            raise ValueError("`task` cannot be None.")
        
        output = program(query = task)
        
        return GetAgentsEvent(task=task, agents=output.agents)

    @step(pass_context=True)
    async def aggregate_results_event(
        self, ctx: Context, ev: GetAgentsEvent
    ) -> AggregateResultsEvent:
        """Step to aggregate results"""
        
        task = ev.task
        agents_to_route = ev.agents
        agents = ctx.agents
        
        responses = []
        
        for agent in agents_to_route:
            response = agents[agent].chat(task)
            responses.append(str(response))
        
        aggregate_result = ". ".join(responses)
        
        return AggregateResultsEvent(task=task, aggregate_result=aggregate_result)
    
    @step(pass_context=True)
    async def consolidate_results_event(
        self, ctx: Context, ev: AggregateResultsEvent
    ) -> StopEvent:
        """Step to consolidate results"""
        
        task = ev.task
        llm = ctx.llm 
        
        ## Format prompt and get result
        
        prompt = CONSOLIDATE_TEMPLATE.format(
            taks=task, aggregated_results = ev.aggregate_result
        )
        
        result = llm.complete(prompt)
        
        return StopEvent(result = result)

In [21]:
from llama_index.core.workflow import draw_all_possible_flows

draw_all_possible_flows(InvestmentWorkflow, filename="InvestmentWorkflow.html")

InvestmentWorkflow.html
